<a target="_blank"  href="https://colab.research.google.com/github/michaelskyuan/Redis-Workshops/blob/main/00-Setup/00-Setup_Redis_Workshops.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Redis Workshop

![Redis](https://redis.com/wp-content/themes/wpx/assets/images/logo-redis.svg?auto=webp&quality=85,75&width=120)

This notebook, as most of the notebooks in this series, is designed to run either with build-in RedisStack or Redis Cloud.

To provision free forever instance of Redis Cloud:
- Head to https://redis.com/try-free/
- Register with email/gmail
- Create **Fixed** subscription with 30MB free tier (no credit card required)
- Create RedisStack Database

In [ ]:
# Install the requirements
%pip install -q redis

### Install Redis Stack locally

Even if you are using Redis Cloud you might need to install Redis Stack just for the cli tool `redis-cli`. 

In [ ]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg 
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list 
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes 


### Setup Redis connection
No changes needed if you are using the in-notebook locally installed Redis Stack. If using Redis Cloud - enter your endpoint host, port, and password here.

In [4]:
import redis
import os
REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
#Replace values above with your own if using Redis Cloud instance
#REDIS_HOST="redis-17231.c228.us-central1-1.gce.cloud.redislabs.com"
#REDIS_PORT=17231
#REDIS_PASSWORD="0XKOePIFBCtuNvV6PhsXl3ysQY6XXXX"

#shortcut for redis-cli $REDIS_CONN command
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

### Basic Test with Redis CLI

In [ ]:
!redis-cli $REDIS_CONN PING
!redis-cli $REDIS_CONN SET hello world
!redis-cli $REDIS_CONN GET hello

### Basic Test with Python client

In [ ]:
r = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD)

r.get("hello")

### TTL - Time To Live

It's extremely important for caching systems to control for how long values will be stored. Redis allows you to control expiry with the EXPIRE command and with flags like EX, EXAT etc. For more information see https://redis.io/commands/expire/ and https://redis.io/commands/set/

In [ ]:
import time
r.set(name="ttl_key", value="this key will expire soon", ex=10)
for i in range(15):
  print(r.get(name="ttl_key"))
  time.sleep(1)